<a href="https://colab.research.google.com/github/Anny-rev/jogo-allura/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [5]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [6]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [7]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta=client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a próxima Imersão IA com Google Gemini. A melhor forma de ficar sabendo é:

*   **Acompanhar as redes sociais da Alura:** Eles geralmente anunciam novos cursos e imersões por lá (LinkedIn, Instagram, etc.).
*   **Assinar a newsletter da Alura:** Assim você recebe novidades diretamente no seu e-mail.
*   **Verificar a página de cursos da Alura:** Fique de olho na página de cursos de IA e Machine Learning para ver se o curso aparece por lá.

Boa sorte!

In [8]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
     config={"tools":[{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 Com base nas informações encontradas, a Alura e o Google realizaram várias edições da "Imersão IA" com o Google Gemini.

A edição mais recente da Imersão IA com Google Gemini que encontrei referência foi uma live de abertura que ocorreu em 9 de maio de 2025. As inscrições para essa imersão estavam abertas até 11 de maio de 2025.

Como as inscrições já se encerraram, não há uma próxima Imersão IA com Google Gemini da Alura com data definida no momento. No entanto, a Alura e o Google realizaram diversas edições desse evento no passado, então é possível que novas edições sejam lançadas no futuro.

In [9]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini da Alura']
Páginas utilizadas na resposta: youtube.com



In [10]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [11]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [12]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [13]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [14]:
##########################################
# --- Agente 1: Buscador de Notícias ---
##########################################
def agente_buscador(topico,data_de_hoje):
    buscador = Agent(
        name= "agente_buscador",
        model= "gemini-2.0-flash",
        description= "Agente que busca no google notícias sobre o tópico indicado",
        tools= [google_search],
        instruction= """Você é um assistente de pesquisa que irá auxiliar uma professora na produção de materiais didáticos da disciplina de história para alunos do ensino fundamental. Você irá utilizar a ferramenta de pesquisa do Google (google_search) para acessar textos relevantes sobre o tópico indicado.
        Foque em no máximo cinco textos relevantes. Considere a respeitabilidade das fontes, dando preferência a textos publicados em sites de universidades e outras instituições respeitadas de produção de conhecimento.
        Descarte textos que geraram muita polêmica ou reações negativas.
        Esses textos devem ser atuais, com publicação no site de no máximo cinco anos da data atual."""
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    textos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return textos_buscados


In [32]:
################################################
# --- Agente 2: Planejador de materiais didáticos --- #
################################################
def agente_planejador(topico, textos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""Você é um planejador na produção de materiais didáticos da disciplina de história para alunos do ensino fundamental. Você é especialista em história.
        Com base nos textos buscados, você deve usar a ferramenta de pesquisa do Google (google_search) para criar um plano que contemple os pontos mais relevantes do topico abordado a serem utilizados na produção de materiais didáticos.
        Você também pode utilizar o (google_search) para buscar mais informações sobre o tópico abordado.
        No final, você irá escolher o ponto mais relevante com base em sua pesquisa, e retornará este ponto com um plano sobre os assuntos a serem abordados no material didático que será produzido posteriormente.""",
        description="Agente que planeja materiais didáticos da disciplina de história",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\ntextos buscados: {textos_buscados}"
    # Executa o agente
    plano_de_material = call_agent(planejador, entrada_do_agente_planejador)
    return plano_de_material

In [33]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_material):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""Você é um produtor de materiais didáticos da disciplina de história para alunos do ensino fundamental.
            Você é um especialista em educação que auxilia professores de história produzindo materiais didáticos.
            Utilize o tema fornecido no plano de material didatico e os pontos mais relevantes fornecidos e, com base nisso,
            produza um rascunho  de material didático sobre o tema indicado. O material produzido deve ser criativo e ser de interesse do público
            a ser contemplado por ele (público pré adolescente). O material deve ser escrito em português.""",
        description="Agente redator de materiais didáticos da disciplina de História para o ensino fundamental"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de material didatico: {plano_de_material}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [18]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de texto meticuloso, especializado na produção de materiais didáticos da disciplina de história para o ensino fundamental.
            Por ter um público jovem, entre 11 e 15 anos, use um tom de escrita e tenha  uma abordagem condizente ao público ao qual o material se destina.
            Revise o rascunho gerado abaixo sobre o tópico indicado, verificando clareza, veracidade das informações, relevância e facilidade de compreensão.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para ser utilizado!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de materiais didáticos da disciplina de história para o ensino fundamental."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [34]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de criação de material didático da disciplina de história para o ensino fundamental 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o seu material ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Por favor, digite um tópico válido")
else:
  print(f"Maravilha! Vamos produzir um excelente material didático sobre {topico} para seus alunos")
  textos_buscados = agente_buscador(topico, data_de_hoje)
  print("/n---Resultado do Agente 1 (Buscador)---/n")
  display(to_markdown(textos_buscados))
  print("----------------------------------------------------")
  plano_do_material =  agente_planejador(topico, textos_buscados);
  print("/n---Resultado do Agente 2 (Planejador)---/n")
  display(to_markdown(plano_do_material))
  rascunho= agente_redator(topico, plano_do_material);
  print("/n---Resultado do Agente 3 (Redator)---/n")
  display(to_markdown(plano_do_material))
  texto_revisado= agente_revisor(topico, rascunho);
  print("/n---Resultado do Agente 4 (Revisor)---/n")
  display(to_markdown(texto_revisado));


🚀 Iniciando o Sistema de criação de material didático da disciplina de história para o ensino fundamental 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o seu material balaiada


Exception in thread Thread-21 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 137, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 125, in _invoke_run_async
    async for event in self.run_async(
  File "/u

Maravilha! Vamos produzir um excelente material didático sobre balaiada para seus alunos
/n---Resultado do Agente 1 (Buscador)---/n


Exception in thread Thread-22 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 137, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 125, in _invoke_run_async
    async for event in self.run_async(
  File "/u

----------------------------------------------------
/n---Resultado do Agente 2 (Planejador)---/n


Exception in thread Thread-23 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 137, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 125, in _invoke_run_async
    async for event in self.run_async(
  File "/u

/n---Resultado do Agente 3 (Redator)---/n


Exception in thread Thread-24 (_asyncio_thread_main):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 137, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
  File "/usr/lib/python3.11/asyncio/runners.py", line 190, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 654, in run_until_complete
    return future.result()
           ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/adk/runners.py", line 125, in _invoke_run_async
    async for event in self.run_async(
  File "/u

/n---Resultado do Agente 4 (Revisor)---/n
